# 0. Initial Setup and Library Imports

In [ ]:
!pip install kagglehub sqlalchemy psycopg2-binary
!pip install sentence-transformers rapidfuzz
!pip install faiss-cpu
import kagglehub
import pandas as pd
import numpy as np
import kagglehub
import os, glob
import csv
from sentence_transformers import SentenceTransformer, util
import torch
from sentence_transformers import SentenceTransformer
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import faiss
from collections import Counter
from tqdm import tqdm


# 1. Data Downloads and Initial Loading


This section describes how raw data from Spotify and Steam is acquired, loaded, and minimally pre-processed before any downstream tasks such as genre resolution, vectorization, and recommendation modeling.
Both datasets are retrieved directly from Kaggle via kagglehub, ensuring reproducibility and consistent file structures across environments.

## 1.1 Spotify Data

Spotify data is sourced from the “900k Spotify Tracks” dataset on Kaggle.
This dataset contains nearly one million audio records with metadata such as genres, duration, and audio features (tempo, energy, valence, etc.).
We download the CSV file and perform an initial inspection to understand column structure.

In [ ]:
spotify_path = kagglehub.dataset_download("devdope/900k-spotify")
print("Spotify files at:", spotify_path)

spotify_files = glob.glob(os.path.join(spotify_path, "*.csv"))
spotify_file = spotify_files[0]

spotify_raw = pd.read_csv(spotify_file, low_memory=False)
spotify_raw.head()

spotify_raw.columns

Using Colab cache for faster access to the '900k-spotify' dataset.
Spotify files at: /kaggle/input/900k-spotify


Index(['Artist(s)', 'song', 'text', 'Length', 'emotion', 'Genre', 'Album',
       'Release Date', 'Key', 'Tempo', 'Loudness (db)', 'Time signature',
       'Explicit', 'Popularity', 'Energy', 'Danceability', 'Positiveness',
       'Speechiness', 'Liveness', 'Acousticness', 'Instrumentalness',
       'Good for Party', 'Good for Work/Study',
       'Good for Relaxation/Meditation', 'Good for Exercise',
       'Good for Running', 'Good for Yoga/Stretching', 'Good for Driving',
       'Good for Social Gatherings', 'Good for Morning Routine',
       'Similar Artist 1', 'Similar Song 1', 'Similarity Score 1',
       'Similar Artist 2', 'Similar Song 2', 'Similarity Score 2',
       'Similar Artist 3', 'Similar Song 3', 'Similarity Score 3'],
      dtype='object')

Description
* Purpose: Load the full Spotify dataset into memory for further cleaning and feature extraction.
* Why low_memory=False: Ensures correct dtype inference given the large number of columns.
* Output: A raw dataframe (spotify_raw) containing track IDs, genres, audio features, and metadata.


## 1.2 Steam Data

Steam game metadata is sourced from the “Steam Games Dataset” on Kaggle.
The original dataset includes a large number of fields, but only a subset is required for this project.
We extract only the essential attributes needed for gameplay modeling, user behavior estimation, and genre mapping.

In [ ]:
csv.field_size_limit(10_000_000)

# ================================================
# 1. Download dataset
# ================================================
steam_path = kagglehub.dataset_download("fronkongames/steam-games-dataset")
steam_file = glob.glob(os.path.join(steam_path, "*.csv"))[0]

# ================================================
# 2. Extract ONLY needed columns:
#    AppID, Name, Median playtime forever, Price, Tags, User score
# ================================================
rows = []

with open(steam_file, "r", encoding="utf-8", errors="ignore") as f:
    reader = csv.reader(f)
    header = next(reader)

    col_AppID   = header.index("AppID")
    col_Name    = header.index("Name")
    col_Median  = header.index("Median playtime forever")
    col_Price   = header.index("Price")
    col_Tags    = header.index("Tags")
    col_UserScr = header.index("User score")      # <-- ★ correctly use "User score"

    max_required = max(col_AppID, col_Name, col_Median, col_Price, col_Tags, col_UserScr)

    for r in reader:
        if len(r) <= max_required:
            continue

        game_id   = r[col_AppID]
        name      = r[col_Name]
        duration  = r[col_Median]
        price     = r[col_Price]
        tags      = r[col_Tags]
        user_scr  = r[col_UserScr]

        rows.append([game_id, name, duration, price, tags, user_scr])

# ================================================
# 3. Build DataFrame (minimal clean version)
# ================================================
steam_df = pd.DataFrame(rows, columns=[
    "game_id",
    "name",
    "duration_minutes",
    "price",
    "genres",
    "user_score"     # <-- ★ replace positive/negative
])

Using Colab cache for faster access to the 'steam-games-dataset' dataset.


Description

* **Purpose:** Extract only the fields required for modeling:
    `AppID`, `Name`, `Playtime`, `Price`, `Tags`, `Positive`, `Negative`, `detailed_description`.
* **Why manual CSV parsing:**
    * Steam CSVs can contain malformed lines or excessively long fields; manual index-based parsing avoids read errors common with `pandas.read_csv()`.
* **Positive / Negative votes:**
    * Added to compute user approval rating (e.g., `rating = pos / (pos + neg)`), a key feature for game vector construction.
* **Tags:**
    * The `Tags` field (not `Genres`) is used, because `Tags` provide more detailed community-driven descriptions.
* **Output:**
    * A clean dataframe (`steam_df`) containing only the columns needed for downstream genre mapping and embedding.

# 2. Data Cleaning and Preprocessing

After loading the raw Spotify and Steam datasets, the next step is to standardize the schema, clean noisy fields, normalize numerical columns, and derive the canonical genre representation (“main genre”) for each game and track.
This preprocessing stage ensures consistency across both domains before performing semantic mapping, embedding generation, and recommendation modeling.

## 2.1 Spotify Data Cleaning

The raw Spotify dataset contains multiple inconsistent formats—mixed string types, irregular timestamps, heterogeneous genre fields, and noisy column naming.
This cleaning pipeline standardizes the schema into a compact, analysis-ready Spotify table.

In [ ]:
spotify_raw = spotify_raw.copy()

# ---- 1) Track ID  ----
spotify_raw["track_id"] = "sp" + spotify_raw.index.astype(str)

# ---- 2) Name & Artists ----
spotify_raw["name"] = spotify_raw["song"].astype(str).str.strip()
spotify_raw["artists"] = spotify_raw["Artist(s)"].astype(str).str.strip()

# ---- 3) Genre ----
spotify_raw["genres"] = spotify_raw["Genre"].astype(str).str.lower().str.strip()

# ---- 4) Length mm:ss → duration_s ----
def mmss_to_seconds(x):
    if isinstance(x, str) and ":" in x:
        m, s = x.split(":")
        return int(m) * 60 + int(s)
    return None

spotify_raw["duration_s"] = spotify_raw["Length"].apply(mmss_to_seconds)

# ---- 5) Standardize numeric audio features ----
num_cols_map = {
    "Tempo": "tempo",
    "Loudness (db)": "loudness_db",
    "Energy": "energy",
    "Danceability": "danceability",
    "Positiveness": "valence",
    "Speechiness": "speechiness",
    "Liveness": "liveness",
    "Acousticness": "acousticness",
    "Instrumentalness": "instrumentalness",
    "Popularity": "popularity"
}

for src, tgt in num_cols_map.items():

    # --- 🔧 minimal fix: extra cleaning for Loudness column ---
    if src == "Loudness (db)":
        cleaned = (
            spotify_raw[src]
            .astype(str)
            .str.replace("dB", "", case=False)
            .str.replace("db", "", case=False)
            .str.replace("[^0-9.\-]", "", regex=True)
        )
        spotify_raw[tgt] = pd.to_numeric(cleaned, errors="coerce")
    else:
        spotify_raw[tgt] = pd.to_numeric(spotify_raw[src], errors="coerce")

# ---- 6) Keep core schema columns ----
spotify_table = spotify_raw[[
    "track_id",
    "name",
    "artists",
    "genres",
    "duration_s",
    "tempo",
    "energy",
    "valence",
    "loudness_db",
    "danceability",
    "instrumentalness",
    "acousticness",
    "popularity"
]].copy()

spotify_table.head()

<>:45: SyntaxWarning: invalid escape sequence '\-'
<>:45: SyntaxWarning: invalid escape sequence '\-'
/tmp/ipython-input-1229701440.py:45: SyntaxWarning: invalid escape sequence '\-'
  .str.replace("[^0-9.\-]", "", regex=True)


,track_id,name,artists,genres,duration_s,tempo,energy,valence,loudness_db,danceability,instrumentalness,acousticness,popularity
0,sp0,Even When the Waters Cold,!!!,hip hop,227,105,83,87,-6.85,71,0,11,40
1,sp1,One Girl / One Boy,!!!,hip hop,243,117,85,87,-5.75,70,0,0,42
2,sp2,Pardon My Freedom,!!!,hip hop,351,121,89,63,-6.06,71,20,0,29
3,sp3,Ooo,!!!,hip hop,224,122,84,97,-5.42,78,0,12,24
4,sp4,Freedom 15,!!!,hip hop,360,123,71,70,-5.57,77,1,4,30


In [ ]:
print([col for col in spotify_raw.columns if "loud" in col.lower()])

['Loudness (db)', 'loudness_db']


## 2.2 Steam Data Cleaning

The Steam dataset includes numeric fields stored as strings, inconsistent genre formats, and requires vote-based rating computation.
This cleaning stage extracts a compact but rich game metadata table.

In [ ]:
# ---------------------------------------------
# Clean numeric
# ---------------------------------------------
steam_df["duration_minutes"] = pd.to_numeric(steam_df["duration_minutes"], errors="coerce").fillna(0)
steam_df["price"] = pd.to_numeric(steam_df["price"], errors="coerce").fillna(0)

# user_score 清洗成数字
steam_df["user_score"] = pd.to_numeric(steam_df["user_score"], errors="coerce")

# ---------------------------------------------
# rating = user_score  （替代 positive/negative 的 rating）
# ---------------------------------------------
steam_df["rating"] = steam_df["user_score"]   # ★ 最小改法：直接赋值

# ---------------------------------------------
# Clean strings
# ---------------------------------------------
steam_df["name"] = steam_df["name"].astype(str).str.strip()
steam_df["genres"] = steam_df["genres"].astype(str).str.strip().str.lower()

# ---------------------------------------------
# Final Steam Table
# ---------------------------------------------
steam_table = steam_df[[
    "game_id",
    "name",
    "rating",               # ★ rating 现在就是 user_score
    "duration_minutes",
    "price",
    "genres"
]].drop_duplicates().reset_index(drop=True)

print("Steam table:", steam_table.shape)
display(steam_table.head())

Steam table: (111452, 6)


,game_id,name,rating,duration_minutes,price,genres
0,20200,Galactic Bowling,NaN,0,19.99,"casual,indie,sports"
1,655370,Train Bandit,NaN,0,0.99,"action,indie"
2,1732930,Jolt Project,NaN,0,4.99,"action,adventure,indie,strategy"
3,1355720,Henosis™,NaN,0,5.99,"adventure,casual,indie"
4,1139950,Two Weeks in Painland,NaN,0,0.00,"adventure,indie"


## 2.3 Derive Main Genres (Steam & Spotify)

Both Steam and Spotify provide multi-tag genre fields, which complicates direct comparison and genre entity mapping.
To produce a unified, canonical representation for each item, we designate one “main genre” per game or track using global tag frequency.

### (A) Main Steam Genre

In [ ]:
# Step 1: Count all Steam tags
steam_tags_all = []
steam_table["genres"].astype(str).apply(
    lambda s: steam_tags_all.extend(
        [t.strip().lower() for t in s.split(",") if t.strip()]
    )
)

tag_freq = Counter(steam_tags_all)

# Step 2:  Select the most representative tag
def pick_main_steam_genre(tag_str):
    tags = [t.strip().lower() for t in tag_str.split(",") if t.strip()]
    if not tags:
        return None
    return max(tags, key=lambda t: tag_freq[t])

steam_table["main_genre"] = steam_table["genres"].apply(pick_main_steam_genre)

### (B) Main Spotify Genre

In [ ]:
from collections import Counter

# Step 1: Collect all Spotify tags
spotify_tags_all = []

spotify_table["genres"].astype(str).apply(
    lambda s: spotify_tags_all.extend(
        [t.strip().lower() for t in s.split(",") if t.strip()]
    )
)

spotify_tag_freq = Counter(spotify_tags_all)

# Step 2: Select the most representative tag
def pick_main_spotify_genre(tag_str):
    tags = [t.strip().lower() for t in tag_str.split(",") if t.strip()]
    if not tags:
        return None
    return max(tags, key=lambda t: spotify_tag_freq[t])

spotify_table["main_genre"] = spotify_table["genres"].apply(pick_main_spotify_genre)
print("Example Spotify main_genre values:")
display(spotify_table[["track_id", "genres", "main_genre"]].head())

Example Spotify main_genre values:


,track_id,genres,main_genre
0,sp0,hip hop,hip hop
1,sp1,hip hop,hip hop
2,sp2,hip hop,hip hop
3,sp3,hip hop,hip hop
4,sp4,hip hop,hip hop


# 3. Genre Entity Resolution and Semantic Mapping

This section reconciles the heterogeneous genre systems used by Steam games and Spotify tracks.
Steam relies on loosely defined, user-generated Tags, while Spotify provides editorially curated textual genre labels.
To enable cross-domain matching and genre-aware recommendations, we standardize, embed, and semantically align all genre tags using Sentence-Transformers.

## 3.1 Steam Genre Tag Extraction & Normalization

Steam genres are stored as comma-separated free-text tags that vary in casing and formatting.
We normalize each entry into a consistent tag list and construct a complete universe of unique Steam genre tags.

In [ ]:
# ================================
# 3.1 Steam: split into tag-level
# ================================

def split_steam_genres(x):
    """Normalize and split Steam genre strings."""
    if not isinstance(x, str):
        return []

    s = x.strip().lower()
    if s == "" or s in ["none", "null", "nan"]:
        return []

    # Comma-separated tags
    tags = [t.strip() for t in s.split(",") if t.strip() != ""]
    return tags

# Apply splitting to the Steam table
steam_table["genre_list"] = steam_table["genres"].apply(split_steam_genres)

# Build Steam tag universe
steam_tag_universe = sorted({t for tags in steam_table["genre_list"] for t in tags})

print("======================================")
print("STEAM tag-level genre statistics")
print("======================================")
print("Number of unique Steam genres:", len(steam_tag_universe))
print("\nExample Steam genres:", steam_tag_universe[:30])

STEAM tag-level genre statistics
Number of unique Steam genres: 33

Example Steam genres: ['360 video', 'accounting', 'action', 'adventure', 'animation & modeling', 'audio production', 'casual', 'design & illustration', 'documentary', 'early access', 'education', 'episodic', 'free to play', 'game development', 'gore', 'indie', 'massively multiplayer', 'movie', 'nudity', 'photo editing', 'racing', 'rpg', 'sexual content', 'short', 'simulation', 'software training', 'sports', 'strategy', 'tutorial', 'utilities']


## 3.2 Spotify Tag Extraction & Normalization

Spotify’s Genre field contains multiple editorial tags per track.
We apply the same normalization strategy to produce a clean Spotify tag universe.

In [ ]:
# ================================
# 3.2 Spotify: split into tag-level
# ================================

def split_spotify_genres(x):
    if not isinstance(x, str) or x.strip() == "":
        return []
    return [t.strip().lower() for t in x.split(",") if t.strip()]

spotify_table["genre_list"] = spotify_table["genres"].apply(split_spotify_genres)

# Build Spotify tag universe
spotify_tag_universe = sorted({t for tags in spotify_table["genre_list"] for t in tags})

print("\n======================================")
print("SPOTIFY tag-level genre statistics")
print("======================================")
print("Number of unique Spotify genres:", len(spotify_tag_universe))
print("Example Spotify genres:", spotify_tag_universe[:30])


SPOTIFY tag-level genre statistics
Number of unique Spotify genres: 88
Example Spotify genres: ['acoustic', 'alt-country', 'alternative', 'alternative rock', 'ambient', 'black metal', 'blues', 'britpop', 'chillout', 'chillwave', 'christian', 'classic rock', 'classical', 'cloud rap', 'comedy', 'country', 'dance', 'dancehall', 'death metal', 'deathcore', 'disco', 'doom metal', 'dream pop', 'drum and bass', 'dub', 'dubstep', 'electro', 'electronic', 'electropop', 'emo']


## 3.3 Sentence-Transformer Embeddings for All Tags

To compare Steam and Spotify genres semantically, we embed all tag strings using a transformer encoder.
We use all-MiniLM-L6-v2 for its efficiency and strong semantic performance.

In [ ]:
from sentence_transformers import SentenceTransformer, util
import torch
import numpy as np
import pandas as pd

# Load transformer encoder once
model = SentenceTransformer("all-MiniLM-L6-v2")

# Encode Steam tags
emb_steam = model.encode(
    steam_tag_universe,
    convert_to_tensor=True,
    normalize_embeddings=True
)

# Encode Spotify tags
emb_spotify = model.encode(
    spotify_tag_universe,
    convert_to_tensor=True,
    normalize_embeddings=True
)

print("Steam emb shape:", emb_steam.shape)
print("Spotify emb shape:", emb_spotify.shape)

Steam emb shape: torch.Size([33, 384])
Spotify emb shape: torch.Size([88, 384])


## 3.4 Semantic Similarity Matrix (Steam → Spotify)

We compute a cosine similarity matrix between every Steam tag embedding and every Spotify tag embedding.
For each Steam tag, we retrieve the Top-K most semantically similar Spotify tags, forming a cross-platform alignment.

In [ ]:
# ================================
# 3.4 Compute semantic similarity matrix
# ================================

sim_matrix = util.cos_sim(emb_steam, emb_spotify)
TOP_K = 10

genre_map_rows = []

for i, st_tag in enumerate(steam_tag_universe):
    top_k = torch.topk(sim_matrix[i], k=TOP_K)

    for score, idx in zip(top_k.values, top_k.indices):
        genre_map_rows.append({
            "steam_tag": st_tag,
            "spotify_tag": spotify_tag_universe[idx],
            "similarity": float(score)
        })

genre_map_table = pd.DataFrame(genre_map_rows)
genre_map_table["similarity"] = genre_map_table["similarity"].round(3)

print("GenreMap shape:", genre_map_table.shape)
genre_map_table.head()

GenreMap shape: (330, 3)


,steam_tag,spotify_tag,similarity
0,360 video,dubstep,0.318
1,360 video,cloud rap,0.311
2,360 video,swing,0.308
3,360 video,dance,0.301
4,360 video,reggae,0.293


## 3.5 Final GenreMap Table (For Database Export)

The final mapping table is sorted by semantic similarity and can serve as a bridge entity between the game and music domains.

In [ ]:
# Final semantic mapping table
genre_map_table = genre_map_table.sort_values(
    ["steam_tag", "similarity"], ascending=[True, False]
).reset_index(drop=True)

display(genre_map_table.head(20))

,steam_tag,spotify_tag,similarity
0,360 video,dubstep,0.318
1,360 video,cloud rap,0.311
2,360 video,swing,0.308
3,360 video,dance,0.301
4,360 video,reggae,0.293
5,360 video,chillwave,0.289
6,360 video,hip hop,0.281
7,360 video,christian,0.272
8,360 video,dub,0.264
9,360 video,j-pop,0.261


# 4. Feature Vector Construction

To enable cross-domain retrieval between games and songs, we construct unified dense vector representations for both Steam games and Spotify tracks.
Each vector integrates normalized numeric features with a genre embedding derived from Sentence-Transformers.
This ensures that recommendations can leverage both behavioral signals (e.g., playtime, audio features) and semantic genre information.

## 4.1 Steam Game Vectors

Each Steam game is represented using three components:
1. Numeric Features
    * User rating
    * Median playtime
    * Price

These are min–max normalized to ensure equal influence.

2. Genre Embedding

We encode each game’s main_genre using the same transformer model used in the semantic mapping stage.

3. Final Game Vector
A 7-dim numeric base vector is concatenated with a 384-dim genre embedding, resulting in a 391-dim feature vector per game.

In [ ]:
# ================================
# 4.1 Steam Game Vector Construction
# ================================
from sentence_transformers import SentenceTransformer
import numpy as np
from sklearn.preprocessing import MinMaxScaler

model = SentenceTransformer("all-MiniLM-L6-v2")

steam = steam_table.copy()

# ---------------------------------------------
# Normalize numeric features
# ---------------------------------------------
steam_numeric = steam[["rating", "duration_minutes", "price"]].fillna(0)
scaler_steam = MinMaxScaler()
steam_numeric_scaled = scaler_steam.fit_transform(steam_numeric)
steam[["rating_n", "duration_minutes_n", "price_n"]] = steam_numeric_scaled

# ---------------------------------------------
# Genre embedding (main_genre)
# ---------------------------------------------
unique_steam_genres = steam["main_genre"].dropna().unique()

steam_genre_emb_map = {
    g: model.encode(g, convert_to_numpy=True).astype("float32").tolist()
    for g in unique_steam_genres
}

steam["genre_emb"] = steam["main_genre"].map(steam_genre_emb_map)

# Fallback for missing embeddings
empty_emb = model.encode("", convert_to_numpy=True)
steam["genre_emb"] = steam["genre_emb"].apply(
    lambda x: empty_emb if isinstance(x, float) or x is None else x
)

# ---------------------------------------------
# Build final game vector
# ---------------------------------------------
def build_game_vec(row):
    # Numeric block: 7 dimensions (3 values + 4 zero padding)
    base7 = np.array([
        row["rating_n"],
        row["duration_minutes_n"],
        row["price_n"],
        0,0,0,0
    ], dtype="float32")

    g_emb = np.array(row["genre_emb"], dtype="float32")
    return np.concatenate([base7, g_emb])  # final = 7 + 384 dims

steam["game_vec"] = steam.apply(build_game_vec, axis=1)

steam_features_pd = steam[["game_id", "main_genre", "game_vec"]].copy()

## 4.2 Spotify Track Vectors

Spotify track vectors integrate:

1. Audio Feature Block Using curated Spotify metadata:
    * tempo, energy, valence
    * danceability, instrumentalness
    * acousticness
    * popularity

  These are min–max normalized.

2. Genre Embedding

Tracks inherit their genre embedding from main_genre, computed similarly to Steam.

3. Final Track Vector

The normalized audio vector (7 dims) concatenated with the 384-dim genre embedding forms a 391-dim track representation.

In [ ]:
# ================================
# 4.2 Spotify Track Vector Construction
# ================================

spotify = spotify_table.copy()

audio_cols = [
    "tempo", "energy", "valence", "danceability",
    "instrumentalness", "acousticness", "popularity"
]

# ---------------------------------------------
# Normalize audio features
# ---------------------------------------------
spotify[audio_cols] = spotify[audio_cols].fillna(0)
scaler_sp = MinMaxScaler()
spotify[audio_cols] = scaler_sp.fit_transform(spotify[audio_cols])

# ---------------------------------------------
# Genre embedding (main_genre)
# ---------------------------------------------
unique_main_genres = spotify["main_genre"].dropna().unique()

spotify_genre_emb_map = {
    g: model.encode(g, convert_to_numpy=True).astype("float32").tolist()
    for g in unique_main_genres
}

spotify["genre_emb"] = spotify["main_genre"].map(spotify_genre_emb_map)

# ---------------------------------------------
# Build final track vector
# ---------------------------------------------
def build_track_vec(row):
    base = row[audio_cols].values.astype("float32")
    g_emb = np.array(row["genre_emb"], dtype="float32")
    return np.concatenate([base, g_emb])

spotify["track_vec"] = spotify.apply(build_track_vec, axis=1)

spotify_features_pd = spotify[["track_id", "main_genre", "track_vec"]].copy()

# 5. Recommendation Engine (FAISS)

To efficiently retrieve the most semantically similar Spotify tracks for each Steam game, we employ FAISS (Facebook AI Similarity Search).
Because both game and track vectors share the same dimensionality and normalization, cosine similarity can be computed as an inner product, enabling scalable approximate nearest-neighbor (ANN) retrieval across millions of items.

## 5.1 FAISS ANN Search

We construct an IVF-Flat index (Inverted File Index) over the Spotify embedding space.
This allows sub-millisecond retrieval per query while maintaining high recall.

Steps Performed

1. Prepare vector matrices for Steam games and Spotify tracks
2. L2 normalize both matrices (cosine → inner product)
3. Initialize IVF index with 4,096 coarse clusters
4. Train the coarse quantizer on 40k sampled Spotify embeddings
5. Add all Spotify vectors to the index
6. Run ANN search for the top-100 nearest tracks per game


In [ ]:
# ================================
# 5.1 FAISS ANN Search
# ================================
import faiss
import numpy as np

# ----------------------------------------------------
# Convert vectors to float32 matrices
# ----------------------------------------------------
steam_vecs = np.stack(steam_features_pd["game_vec"].values).astype("float32")
spotify_vecs = np.stack(spotify_features_pd["track_vec"].values).astype("float32")

# ----------------------------------------------------
# Normalize for cosine similarity (inner product)
# ----------------------------------------------------
faiss.normalize_L2(steam_vecs)
faiss.normalize_L2(spotify_vecs)

# ----------------------------------------------------
# Build IVF index
# ----------------------------------------------------
d = spotify_vecs.shape[1]       # vector dimension
nlist = 4096                    # number of coarse clusters

quantizer = faiss.IndexFlatIP(d)
index = faiss.IndexIVFFlat(quantizer, d, nlist, faiss.METRIC_INNER_PRODUCT)

# ----------------------------------------------------
# Train index on subset of Spotify vectors
# ----------------------------------------------------
np.random.seed(42)
sample_size = min(40000, len(spotify_vecs))
sample_idx = np.random.choice(len(spotify_vecs), sample_size, replace=False)

index.train(spotify_vecs[sample_idx])
print("IVF trained on sample subset:", sample_size)

# ----------------------------------------------------
# Add all Spotify vectors to IVF index
# ----------------------------------------------------
index.add(spotify_vecs)
print("Index size:", index.ntotal)

# ----------------------------------------------------
# Search top-100 nearest tracks for each game
# ----------------------------------------------------
index.nprobe = 16
TOP_K = 100

scores, indices = index.search(steam_vecs, TOP_K)

IVF trained on sample subset: 40000
Index size: 551443


## 5.2 Build Recommendations Table

Each Steam game receives its TOP 100 Spotify tracks, ranked by semantic similarity.
The output is denormalized into a recommendation table

In [ ]:
# ================================
# 5.2 Assemble Recommendations Table
# ================================
rows = []
track_ids = spotify_features_pd["track_id"].values
game_ids = steam_features_pd["game_id"].values

for i, game_id in enumerate(game_ids):
    for k in range(TOP_K):
        track_idx = indices[i, k]
        score = float(scores[i, k])

        rows.append({
            "game_id": int(game_id),
            "track_id": track_ids[track_idx],   # track_id is string (e.g., "sp123456")
            "match_score": round(score, 4)
        })

recommendations_table = pd.DataFrame(rows)
print("Recommendations:", recommendations_table.shape)
display(recommendations_table.head())

Recommendations: (11145200, 3)


,game_id,track_id,match_score
0,20200,sp109500,0.8621
1,20200,sp43412,0.8421
2,20200,sp25343,0.8326
3,20200,sp294406,0.8260
4,20200,sp40477,0.8161


# 6. Derived Tables for Recommendation System

## 6.1 GameGen Table

This section constructs several supporting tables used by the downstream recommendation engine and database schema, including GameGen, MusicGen, GenMap, Playlist, Contains, and user-related tables. These tables decouple entities (games, tracks, genres) and support efficient querying via foreign keys.

In [ ]:
# ================================
# 6.1 GameGen table
# ================================

game_gen = steam_table[["game_id", "main_genre"]].rename(
    columns={"main_genre": "game_genre"}
).dropna().drop_duplicates()

print("GameGen:", game_gen.shape)
display(game_gen.head())

GameGen: (105012, 2)


,game_id,game_genre
0,20200,indie
1,655370,indie
2,1732930,indie
3,1355720,indie
4,1139950,indie


## 6.2 MusicGen Table

Similar to Steam, each Spotify track is assigned a primary genre (highest-frequency tag globally).
This forms a reference table connecting each track to the single genre used in the semantic mapping and playlist generation.


In [ ]:
# =====================================================
# 6.2 MusicGen table
# =====================================================

music_gen = spotify_table[["track_id", "main_genre"]].drop_duplicates()

music_gen = music_gen.rename(columns={
    "main_genre": "track_genre"
})

music_gen["track_id"] = music_gen["track_id"].astype(str)
music_gen["track_genre"] = music_gen["track_genre"].astype(str)

print("MusicGen table:", music_gen.shape)
display(music_gen.head())

MusicGen table: (551443, 2)


,track_id,track_genre
0,sp0,hip hop
1,sp1,hip hop
2,sp2,hip hop
3,sp3,hip hop
4,sp4,hip hop


## 6.3 GenMap Table

The GenMap table expresses cross-domain semantic similarity between Steam and Spotify genres,
built earlier using SentenceTransformer embeddings + cosine similarity.

Only the (`game_genre`, `track_genre`) pairs are retained for database storage.

In [ ]:
# =====================================================
# 6.3 GenMap table
# =====================================================

gen_map = genre_map_table.rename(columns={
    "steam_tag": "game_genre",
    "spotify_tag": "track_genre"
})[["game_genre", "track_genre"]].drop_duplicates().reset_index(drop=True)

print("GenMap:", gen_map.shape)
display(gen_map.head())

GenMap: (330, 2)


,game_genre,track_genre
0,360 video,dubstep
1,360 video,cloud rap
2,360 video,swing
3,360 video,dance
4,360 video,reggae


## 6.4 Playlist & Contains Tables

To produce human-interpretable playlist outputs, one playlist is created per Steam genre, populated with the top-K (1000) Spotify tracks from the recommendation engine.

* Playlist: Metadata for each generated playlist
* Contains: A many-to-many table linking playlists to individual tracks

In [ ]:
# =====================================================
# 6.4 Playlist + Contains tables
# =====================================================

TRACKS_PER_PLAYLIST = 1000

playlist_rows = []
contains_rows = []

# Unique game genres
genres = game_gen["game_genre"].unique()

pid_counter = 0

# Normalize dtypes
recommendations_table["game_id"] = recommendations_table["game_id"].astype(int)
game_gen["game_id"] = game_gen["game_id"].astype(int)
recommendations_table["track_id"] = recommendations_table["track_id"].astype(str)
spotify_table["track_id"] = spotify_table["track_id"].astype(str)

# Merge rec table with GameGen (for grouping by genre)
rec_merged = recommendations_table.merge(
    game_gen,
    on="game_id",
    how="inner"
)

# Spotify durations
spotify_dur = spotify_table[["track_id", "duration_s"]].copy()
spotify_dur["duration_minutes"] = spotify_dur["duration_s"] / 60.0

# Construct playlist per genre
for g in genres:

    pid = f"pl{pid_counter}"
    playlist_name = f"{g.capitalize()} Mood Playlist"

    subset = rec_merged[rec_merged["game_genre"] == g]

    if subset.empty:
        pid_counter += 1
        continue

    top_n = subset.sort_values("match_score", ascending=False).head(TRACKS_PER_PLAYLIST)
    merged = top_n.merge(spotify_dur, on="track_id", how="left")
    total_duration = int(merged["duration_minutes"].sum())

    # Playlist row
    playlist_rows.append({
        "playlist_id": pid,
        "playlist_name": playlist_name,
        "total_duration_minutes": total_duration,
        "total_tracks": len(merged)
    })

    # Contains rows
    for tid in merged["track_id"]:
        contains_rows.append({
            "playlist_id": pid,
            "track_id": tid
        })

    pid_counter += 1

playlist = pd.DataFrame(playlist_rows)
contains = pd.DataFrame(contains_rows)

print("Playlist:", playlist.shape)
print("Contains:", contains.shape)
display(playlist.head())
display(contains.head())

Playlist: (24, 4)
Contains: (22800, 2)


,playlist_id,playlist_name,total_duration_minutes,total_tracks
0,pl0,Indie Mood Playlist,2393,1000
1,pl1,Casual Mood Playlist,5145,1000
2,pl2,Adventure Mood Playlist,2136,1000
3,pl3,Strategy Mood Playlist,3315,1000
4,pl4,Action Mood Playlist,4675,1000


,playlist_id,track_id
0,pl0,sp43337
1,pl0,sp193515
2,pl0,sp402072
3,pl0,sp193521
4,pl0,sp109500


## 6.5 User, Saved, and Owned Tables (Initialization)

The system includes three additional relational tables for user-specific personalization:

* Users: Core user metadata (empty at initialization)
* Saved: Many-to-many table of saved playlists
* Owned: Many-to-many table of Steam games owned by a user

These tables start empty to be populated by the front-end application.

In [ ]:
# =====================================================
# 6.5 User, Saved, Owned tables
# =====================================================

# Users table (empty)
user_cols = ["user_id", "spotify_id", "spotify_name", "steam_id", "steam_name"]
users = pd.DataFrame(columns=user_cols)
print(users)

# Saved playlists (empty)
saved = pd.DataFrame(columns=["user_id", "playlist_id"])
print(saved)

# Owned games (empty)
owned = pd.DataFrame({
    "user_id": pd.Series(dtype="int"),
    "game_id": pd.Series(dtype="int")
})
print(owned)

Empty DataFrame
Columns: [user_id, spotify_id, spotify_name, steam_id, steam_name]
Index: []
Empty DataFrame
Columns: [user_id, playlist_id]
Index: []
Empty DataFrame
Columns: [user_id, game_id]
Index: []


# 7. Export All Tables to CSV

After constructing all derived tables (Steam, Spotify, genre mappings, recommendations, playlists, and user metadata), the final step is to standardize schema types and export all tables to CSV files.
This ensures compatibility with downstream relational databases (MySQL, Postgres, SQLite) and avoids foreign-key mismatches due to mixed datatypes.

Before exporting, all identifier fields (game_id, track_id, playlist_id) and genre columns must be normalized.
This prevents common integration failures such as:
* int vs string mismatches
* NaN values turning into "nan"
* inconsistent casing or Unicode issues

## 7.1 Standardize Key Column Types

In [ ]:
# ===================================================
# 1. Standardize datatypes for all key columns
# ===================================================

# ---- Game-related IDs ----
if "game_id" in steam_table.columns:
    steam_table["game_id"] = steam_table["game_id"].astype(int)

if "game_id" in game_gen.columns:
    game_gen["game_id"] = game_gen["game_id"].astype(int)

if "game_id" in recommendations_table.columns:
    recommendations_table["game_id"] = recommendations_table["game_id"].astype(int)

if "game_id" in owned.columns:
    owned["game_id"] = owned["game_id"].astype(int)

# ---- Music-related IDs ----
if "track_id" in spotify_table.columns:
    spotify_table["track_id"] = spotify_table["track_id"].astype(str)

if "track_id" in music_gen.columns:
    music_gen["track_id"] = music_gen["track_id"].astype(str)

if "track_id" in recommendations_table.columns:
    recommendations_table["track_id"] = recommendations_table["track_id"].astype(str)

if "track_id" in contains.columns:
    contains["track_id"] = contains["track_id"].astype(str)

if "track_id" in saved.columns:
    saved["track_id"] = saved["track_id"].astype(str)

# ---- Playlist IDs ----
if "playlist_id" in playlist.columns:
    playlist["playlist_id"] = playlist["playlist_id"].astype(str)

if "playlist_id" in contains.columns:
    contains["playlist_id"] = contains["playlist_id"].astype(str)

# ---- Normalize all genre columns as strings ----
genre_columns = ["genre", "game_genre", "track_genre", "main_genre"]

for df in [steam_table, spotify_table, game_gen, music_gen, gen_map]:
    for col in genre_columns:
        if col in df.columns:
            df[col] = df[col].astype(str)

## Export Table Dictionary

We collect all final DataFrames into a single dictionary to simplify export and ensure reproducibility.

In [ ]:
# ===================================================
# 2. Tables to export
# ===================================================
tables = {
    "steam": steam_table,
    "spotify": spotify_table,
    "game_gen": game_gen,
    "music_gen": music_gen,
    "gen_map": gen_map,
    "recommendations": recommendations_table,
    "playlist": playlist,
    "contains": contains,
    "users": users,
    "saved": saved,
    "owned": owned
}

## 7.3 Export CSV Files with Progress Bar

Using tqdm provides visibility into long exports, especially when writing multi-million-row tables such as the Recommendations table.

In [ ]:
# ===================================================
# 3. Export all tables as CSV (with tqdm)
# ===================================================
from tqdm import tqdm

for name, df in tqdm(tables.items(), desc="Exporting tables"):
    df = df.copy()  # defensive copy
    df.to_csv(f"{name}.csv", index=False)

Exporting tables: 100%|██████████| 11/11 [00:31<00:00,  2.89s/it]
